In [10]:
import numpy as np
import pandas as pd

import os
import pickle
import glob
import matplotlib.pyplot as plt
import time
import keras
import imageio
from scipy import signal
from keras.models import load_model
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import explained_variance_score
from sklearn.metrics import roc_curve, auc
from skimage.transform import resize
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


In [11]:
save_figures = False
file_ending = '.png'
model_string = 'NMDA'

dataset_folder = '../data'

models_folder     = os.path.join(dataset_folder, 'Models')
morphology_folder = os.path.join(dataset_folder, 'Morphology')
test_data_folder  = os.path.join(dataset_folder, 'Data_test')
#auxiliary_folder  = os.path.join(dataset_folder, 'Auxiliary')

model_filename           = os.path.join(models_folder, 'NMDA_TCN__DWT_7_128_153__model.h5')
model_metadata_filename  = os.path.join(models_folder, 'NMDA_TCN__DWT_7_128_153__training.pickle')
morphology_filename      = os.path.join(morphology_folder, 'morphology_dict.pickle')
#NN_illustration_filename = os.path.join(auxiliary_folder, 'TCN_7_layers.png')
test_files               = sorted(glob.glob(os.path.join(test_data_folder, '*_128_simulationRuns*_6_secDuration_*')))

print('-----------------------------------------------')
print('finding files: morphology and test data')
print('-----------------------------------------------')
#print('model found          : "%s"' %(model_filename.split('/')[-1]))
print('model metadata found : "%s"' %(model_metadata_filename.split('/')[-1]))
print('morphology found     : "%s"' %(morphology_filename.split('/')[-1]))
print('number of test files is %d' %(len(test_files)))
print('-----------------------------------------------')

-----------------------------------------------
finding files: morphology and test data
-----------------------------------------------
model metadata found : "NMDA_TCN__DWT_7_128_153__training.pickle"
morphology found     : "morphology_dict.pickle"
number of test files is 12
-----------------------------------------------


In [14]:
testing_3 = test_files[0]
experiment_dict = pickle.load(open(testing_3, "rb" ), encoding='latin1')


In [15]:
experiment_dict.keys()
df = pd.DataFrame(experiment_dict)

In [38]:
opening_files = test_files[:3]

In [166]:
def dataframe(files):
    for i in files:
        experiment_dict = pickle.load(open(testing_3, "rb" ), encoding='latin1')
        df = pd.DataFrame(experiment_dict)
        files.drop('Results', inplace=True, axis=1)
        files.drop(['useActiveDendrites', 'randomSeed', 'averageOutputFrequency', 'allSectionsType', 'allSectionsLength', 'allSections_DistFromSoma', 'listOfISIs', 'inst_rate_sampling_time_interval_options_ms' 'numOutputSpikesPerSim', 'collectAndSaveDVTs', 'numSimulations', 'totalSimDurationInSec', 'num_bas_ex_spikes_per_100ms_range', 'totalNumSimulationSeconds', 'ISICV', 'inhibitorySynapseType', 'excitatorySynapseType', 'num_apic_ex_inh_spike_diff_per_100ms_range', 'numSamplesPerMS_HighRes', 'entireExperimentDurationInMinutes', 'Ih_vshift', 'num_bas_ex_inh_spike_diff_per_100ms_range', 'totalNumOutputSpikes', 'num_apic_ex_spikes_per_100ms_range', 'listOfSingleSimulationDicts'], axis=0, inplace=True)
    
    return pd.concat(files)

df_sub = df.loc[['allSectionsType', 'allSectionsLength', 'allSections_DistFromSoma']]

df_sub.T.apply(pd.Series.explode).reset_index()
# stratify on column for basal/apical - use labelencoder

index allSectionsType allSectionsLength allSections_DistFromSoma
0    Params           basal           24.2009                  11.5847
1    Params           basal           15.1822                  35.7856
2    Params           basal           151.651                  50.9678
3    Params           basal           7.45563                  50.9678
4    Params           basal           92.5758                  58.4234
..      ...             ...               ...                      ...
188  Params          apical           23.3443                  57.7203
189  Params          apical           93.6788                  81.0646
190  Params          apical           22.0852                  81.0646
191  Params          apical           36.0978                   103.15
192  Params          apical           30.0747                   103.15

[193 rows x 4 columns]

In [141]:
df
lst = list((df.loc['numOutputSpikesPerSim']))[0]

df1 = pd.DataFrame({'numOutputSpikesPerSim':lst})
df.Params.apply(lambda x: len(x))


numOutputSpikesPerSim                          128
allSegmentsLength                              639
allSegments_SectionInd                         639
allSegments_SectionDistFromSoma                639
allSectionsType                                193
allSectionsLength                              193
allSections_DistFromSoma                       193
allSegments_DistFromSoma                       639
allSegmentsType                                639
listOfISIs                                     494
inst_rate_sampling_time_interval_options_ms     18
Name: Params, dtype: int64

In [161]:
files = df_sub

final_df = pd.DataFrame()

for index, row in files.iteritems():
    lst1 = files.loc(row)
    df_file = pd.DataFrame(lst1)
    print(df_file)
    final_df.append(df_file)



TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [ ]:
'''##%% helper functions

def bin2dict(bin_spikes_matrix):
    spike_row_inds, spike_times = np.nonzero(bin_spikes_matrix)
    row_inds_spike_times_map = {}
    for row_ind, syn_time in zip(spike_row_inds,spike_times):
        if row_ind in row_inds_spike_times_map.keys():
            row_inds_spike_times_map[row_ind].append(syn_time)
        else:
            row_inds_spike_times_map[row_ind] = [syn_time]

    return row_inds_spike_times_map


def dict2bin(row_inds_spike_times_map, num_segments, sim_duration_ms):
    
    bin_spikes_matrix = np.zeros((num_segments, sim_duration_ms), dtype='bool')
    for row_ind in row_inds_spike_times_map.keys():
        for spike_time in row_inds_spike_times_map[row_ind]:
            bin_spikes_matrix[row_ind,spike_time] = 1.0
    
    return bin_spikes_matrix'''

In [7]:
'''experiment_dict = pickle.load(open(test_files[0], "rb" ), encoding='latin1')

# gather params
num_simulations = len(experiment_dict['Results']['listOfSingleSimulationDicts'])
num_segments_basal    = len(experiment_dict[experiment_dict['Params']['allSegmentsType'] == 'basal'])
num_segments_apical    = len(experiment_dict[experiment_dict['Params']['allSegmentsType'] == 'apical'])
sim_duration_ms = experiment_dict['Params']['totalSimDurationInSec'] * 1000
num_ex_synapses_b  = num_segments_basal /2
num_inh_synapses_b = num_segments_basal/2 
num_ex_synapses_a  = num_segments_apical/2 
num_inh_synapses_a = num_segments_apical/2
num_synapses_b = num_ex_synapses_b + num_inh_synapses_b
num_synapses_a = num_ex_synapses_a + num_inh_synapses_a''';

KeyError: False